In [1]:
# On affiche les graphiques dans le notebook en statique
%matplotlib inline

In [2]:
import pandas as pd
import cupy as cp
import cudf
import dask_cudf
import cuml
from cuml.model_selection import train_test_split
import sys
import os
import time
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [3]:
from src.p7_constantes import DATA_INTERIM
from src.p7_util import format_time, timer

In [4]:
%load_ext autoreload
%autoreload 2
#autoreload 2 = Reload all modules (except those excluded by %aimport)
# every time before executing the Python code typed.

# Versions

In [5]:
print(f"Version Pandas : {pd.__version__}")
print(f"Version numpy : {np.__version__}")
print(f"Version cuDF : {cudf.__version__}")
print(f"Version cuML : {cuml.__version__}")

Version Pandas : 2.2.1
Version numpy : 1.26.4
Version cuDF : 24.04.01
Version cuML : 24.04.00


In [6]:
print("Path ptyhon Env virtuelavec pyenv :", sys.executable)
print("Version de Python :", sys.version)

Path ptyhon Env virtuelavec pyenv : /home/arianewsl/.pyenv/versions/3.10.6/envs/wp7/bin/python
Version de Python : 3.10.6 (main, May 23 2024, 11:14:43) [GCC 11.4.0]


# Utilisation pandas

In [7]:
df = pd.read_csv(os.path.join(DATA_INTERIM, "v1_train.csv")).drop("Unnamed: 0", axis=1)
df.shape

(230633, 757)

In [8]:
df.head(2)

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Refused_SUM,CC_NAME_CONTRACT_STATUS_Refused_VAR,CC_NAME_CONTRACT_STATUS_Sentproposal_MEAN,CC_NAME_CONTRACT_STATUS_Sentproposal_SUM,CC_NAME_CONTRACT_STATUS_Sentproposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_COUNT,TARGET
0,199392,0.0,1,1,1,225000.0,270000.0,13500.0,270000.0,0.008625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,346123,1.0,0,0,0,540000.0,2013840.0,53253.0,1800000.0,0.016612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


# Utilisation GPU pour corr

In [9]:
"""
https://learn.microsoft.com/en-us/windows/wsl/tutorials/gpu-compute
prerequisites : Setting up NVIDIA CUDA with Docker !
"""

'\nhttps://learn.microsoft.com/en-us/windows/wsl/tutorials/gpu-compute\nprerequisites : Setting up NVIDIA CUDA with Docker !\n'

test cudf

In [10]:
df = cudf.DataFrame(
    {
        "a": list(range(20)),
        "b": list(reversed(range(20))),
        "c": list(range(20)),
    }
)
df

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2
3,3,16,3
4,4,15,4
5,5,14,5
6,6,13,6
7,7,12,7
8,8,11,8
9,9,10,9


In [11]:
df.corr()

,a,b,c
a,1.0,-1.0,1.0
b,-1.0,1.0,-1.0
c,1.0,-1.0,1.0


Essai avec gros Dataset cuDF

In [12]:
df = cudf.read_csv("data/interim/v1_train.csv").drop("Unnamed: 0", axis=1)
df.shape

(230633, 757)

In [14]:
df = df.fillna(1)

In [15]:
t0 = time.time()
correlations = df.corr(method='spearman')
duration = time.time() - t0
print("Durée avec cudf =", format_time(duration))

Durée avec cudf = 0:00:04


In [76]:
type(correlations)

cudf.core.dataframe.DataFrame

In [77]:
type(correlations.to_numpy())

numpy.ndarray

In [16]:
corr_array = correlations.to_numpy()

In [17]:
#corr_ceiled = corr_ceiled.replace(to_replace=1.0, value=0)
np.fill_diagonal(corr_array, 0.0)
corr_array

array([[ 0.        , -0.00129011,  0.00101939, ..., -0.00150814,
         0.00116338, -0.00109045],
       [-0.00129011,  0.        , -0.34768136, ..., -0.01660667,
         0.01625268, -0.05391229],
       [ 0.00101939, -0.34768136,  0.        , ...,  0.01131404,
        -0.00645778, -0.02061576],
       ...,
       [-0.00150814, -0.01660667,  0.01131404, ...,  0.        ,
        -0.97891459, -0.01421914],
       [ 0.00116338,  0.01625268, -0.00645778, ..., -0.97891459,
         0.        ,  0.00776676],
       [-0.00109045, -0.05391229, -0.02061576, ..., -0.01421914,
         0.00776676,  0.        ]])

In [18]:
corr_cudf = cudf.DataFrame(corr_array, columns=correlations.columns)

In [19]:
corr_ceiled = corr_cudf[corr_cudf > 0.9]
corr_ceiled.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Refused_SUM,CC_NAME_CONTRACT_STATUS_Refused_VAR,CC_NAME_CONTRACT_STATUS_Sentproposal_MEAN,CC_NAME_CONTRACT_STATUS_Sentproposal_SUM,CC_NAME_CONTRACT_STATUS_Sentproposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_COUNT,TARGET
0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [20]:
corr_ceiled.dropna(axis=1, how="all", inplace=True)
corr_ceiled.dropna(axis=0, how="all", inplace=True)

In [21]:
corr_ceiled

,AMT_CREDIT,AMT_GOODS_PRICE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,...,CC_NAME_CONTRACT_STATUS_Refused_MEAN,CC_NAME_CONTRACT_STATUS_Refused_SUM,CC_NAME_CONTRACT_STATUS_Refused_VAR,CC_NAME_CONTRACT_STATUS_Sentproposal_MEAN,CC_NAME_CONTRACT_STATUS_Sentproposal_SUM,CC_NAME_CONTRACT_STATUS_Sentproposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_COUNT
6,<NA>,0.984472427,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,0.984472427,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21,<NA>,<NA>,<NA>,0.949935616,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
22,<NA>,<NA>,0.949935616,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
33,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.994644504,0.994525191,0.999697571,0.99493366,0.992971453,<NA>,0.992114026,0.94447554,0.997154132,<NA>
752,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.997128207,0.997011842,0.992400704,0.996836828,0.993388028,0.992114026,<NA>,0.969198798,0.994993897,<NA>
753,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.949679749,0.949583454,0.944742173,0.949409189,0.946157694,0.94447554,0.969198798,<NA>,0.963812646,<NA>
754,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.992400367,0.992283271,0.997442025,0.992096237,0.988626391,0.997154132,0.994993897,0.963812646,<NA>,<NA>


Essai avec gros Dataset Pandas sans NaN

In [22]:
df = pd.read_csv("data/interim/v1_train.csv").drop("Unnamed: 0", axis=1)
df.shape

(230633, 757)

In [23]:
df = df.fillna(1)

In [24]:
t0 = time.time()
correlations = df.corr(method='spearman')
duration = time.time() - t0
print("Durée Pandas =", format_time(duration))

Durée Pandas = 0:04:50


In [ ]:
corr_ceiled = correlations[correlations > 0.5].head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Refused_SUM,CC_NAME_CONTRACT_STATUS_Refused_VAR,CC_NAME_CONTRACT_STATUS_Sentproposal_MEAN,CC_NAME_CONTRACT_STATUS_Sentproposal_SUM,CC_NAME_CONTRACT_STATUS_Sentproposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_COUNT,TARGET
SK_ID_CURR,1.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
CODE_GENDER,<NA>,1.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
FLAG_OWN_CAR,<NA>,<NA>,1.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
FLAG_OWN_REALTY,<NA>,<NA>,<NA>,1.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
CNT_CHILDREN,<NA>,<NA>,<NA>,<NA>,1.0,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
correlations

Essai avec Pandas et NaN

In [25]:
df = pd.read_csv("data/interim/v1_train.csv").drop("Unnamed: 0", axis=1)
df.shape
t0 = time.time()
correlations = df.corr(method='spearman')
duration = time.time() - t0
print("Durée Pandas avec NaN=", format_time(duration))

Durée Pandas avec NaN= 0:40:48


# essai train test split sur cudf

## train_test_split sans NaN

In [87]:
df = cudf.read_csv("data/interim/v1_train.csv").drop("Unnamed: 0", axis=1)
df.shape

(230633, 757)

In [92]:
print(df.dtypes.value_counts())

float64    580
bool       136
int64       41
Name: count, dtype: int64


In [93]:
# Remplacer NaN pour les colonnes de type int par le mode
for col in df.select_dtypes(include=['int64', 'bool']):
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)

# Remplacer NaN pour les colonnes de type float par la médiane
for col in df.select_dtypes(include=['float64']):
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)


In [97]:
df.isna().sum()

SK_ID_CURR                             0
CODE_GENDER                            0
FLAG_OWN_CAR                           0
FLAG_OWN_REALTY                        0
CNT_CHILDREN                           0
                                      ..
CC_NAME_CONTRACT_STATUS_Signed_MEAN    0
CC_NAME_CONTRACT_STATUS_Signed_SUM     0
CC_NAME_CONTRACT_STATUS_Signed_VAR     0
CC_COUNT                               0
TARGET                                 0
Length: 757, dtype: int64

In [99]:
type(df)

cudf.core.dataframe.DataFrame

In [100]:
# Pas de stratify sous cette forme, il faut utiliser la forme détaillée
cu_train, cu_val = train_test_split(df, test_size=0.25)

In [107]:
cu_train.shape

(172975, 757)

In [101]:
predictors = [col for col in df.columns if col not in ['TARGET', 'SK_ID_CURR']]
X = df[predictors]
y = df['TARGET']

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

In [106]:
X_test.shape

(57658, 755)

In [108]:
X_train.shape

(172975, 755)

## train_test_split AVEC NaN

In [8]:
df = cudf.read_csv("data/interim/v1_train.csv").drop("Unnamed: 0", axis=1)
df.shape

(230633, 757)

In [9]:
print(df.dtypes.value_counts())

float64    580
bool       136
int64       41
Name: count, dtype: int64


In [10]:
df.isna().sum().sum()

43846728

In [11]:
type(df)

cudf.core.dataframe.DataFrame

In [12]:
predictors = [col for col in df.columns if col not in ['TARGET', 'SK_ID_CURR']]
X = df[predictors]
y = df['TARGET']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

ValueError: Column must have no nulls.

In [ ]:
"""
Train test split sur cuDF n'accepte pas les NaN
"""

(57658, 755)

# Pandas et cuDF : NaN is a float 

Lecture des données

In [11]:
with timer("Lecture avec cuDF"):
    df_cu = cudf.read_csv("data/interim/v1_train.csv").drop("Unnamed: 0", axis=1)
    print("shape ", df_cu.shape)
    print("Nombre de NaN :", df_cu.isna().sum().sum())
    print("df est de type :", type(df_cu))
    print("\nRépartition des types :")
    print(df_cu.dtypes.value_counts())
    print("\nInfo :")
    df_cu.info()

shape  (230633, 757)
Nombre de NaN : 43846728
df est de type : <class 'cudf.core.dataframe.DataFrame'>

Répartition des types :
float64    580
bool       136
int64       41
Name: count, dtype: int64

Info :
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 230633 entries, 0 to 230632
Columns: 757 entries, SK_ID_CURR to TARGET
dtypes: bool(136), float64(580), int64(41)
memory usage: 1.1 GB
Lecture avec cuDF - duration (hh:mm:ss) : 0:00:01


In [12]:
with timer("Lecture avec pandas"):
    df_pd = pd.read_csv("data/interim/v1_train.csv").drop("Unnamed: 0", axis=1)
    print("shape ", df_pd.shape)
    print("Nombre de NaN :", df_pd.isna().sum().sum())
    print("df est de type :", type(df_pd))
    print("\nRépartition des types :")
    print(df_pd.dtypes.value_counts())
    print("\nInfo :")
    df_pd.info()

shape  (230633, 757)
Nombre de NaN : 43846728
df est de type : <class 'pandas.core.frame.DataFrame'>

Répartition des types :
float64    580
bool       136
int64       41
Name: count, dtype: int64

Info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230633 entries, 0 to 230632
Columns: 757 entries, SK_ID_CURR to TARGET
dtypes: bool(136), float64(580), int64(41)
memory usage: 1.1 GB
Lecture avec pandas - duration (hh:mm:ss) : 0:00:07


Ajout de NaN dans les Int / bool

In [19]:
columns_int = df_pd.select_dtypes(include="int64").columns
columns_bool = df_pd.select_dtypes(include="bool").columns

In [20]:
df_pd.select_dtypes(include="int64").isna().sum().sum()

0.0

In [21]:
df_pd['CODE_GENDER'].dtypes

dtype('float64')

In [25]:
# On place des NaN dans la première ligne des columns int et bool
idx = df_pd.head(1).index
df_pd.loc[idx, columns_int] = np.nan
df_pd.loc[idx, columns_bool] = np.nan

In [26]:
# Nouvelle répartition des types
print(df_pd.dtypes.value_counts())
# Nouvelles info
print("\nInfo :")
df_pd.info()

float64    621
object     136
Name: count, dtype: int64

Info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230633 entries, 0 to 230632
Columns: 757 entries, SK_ID_CURR to TARGET
dtypes: float64(621), object(136)
memory usage: 1.3+ GB


In [ ]:
"""
Avant d'exploiter le df, il faut transformer les bool en int
Toutes les colonnes int contenant des NaN sont transformées en float64
Toutes les colonnes restées en bool mais contenant des NaN sont transformées en Object
Avec une répercussion sur la memory usage
"""

# Essai light GBM

In [110]:
"""import lightgbm as lgb
from lightgbm import LGBMClassifier"""

In [111]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": ["binary_logloss", "auc"],
    
    "num_leaves": 34,
    "learning_rate": 0.05,
    #"colsample_bytree":0.9497036,
    "max_depth": 8,
    "reg_alpha": 0.041545473,
    "reg_lambda": 0.0735294,
    "min_split_gain": 0.0222415,
    "min_child_weight": 39.3259775,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "num_threads": 32,
    "verbose": -1, # Supprime la plupart des warnings lgbm
}
verbose = True

In [114]:
# Conversion en dataset lgbm
lgb_train = lgb.Dataset(X_train.to_pandas(), y_train.to_pandas(), free_raw_data=True)
lgb_eval = lgb.Dataset(X_test.to_pandas(), y_test.to_pandas(), reference=lgb_train, free_raw_data=True)

metrics = {}
if verbose:
    callbacks = [
        lgb.early_stopping(stopping_rounds=50),
        # lgb.log_evaluation(period=50),
        lgb.record_evaluation(metrics),
    ]
    print("Train shape :", X_train.shape)
    print("Eval shape :", X_test.shape)
    print("Nombre de prédicteurs :", len(predictors))
    #print("Nombre de features catégorielles :", len(categorical_features))
    print("\nEntraînement...")
else:
    callbacks = [
        lgb.early_stopping(stopping_rounds=50),
        lgb.record_evaluation(metrics),
    ]
t0_fit = time.time()
lgb_clf = lgb.train(
    params,
    lgb_train,
    num_boost_round=400,  # Correspond à n_estimators max
    valid_sets=[lgb_train, lgb_eval],
    valid_names=["train", "eval"],
    feature_name=predictors,
    #categorical_feature=categorical_features,
    callbacks=callbacks,
)

fit_duration = time.time() - t0_fit
if verbose:
    print("Durée du fit (hh:mm:ss):", format_time(fit_duration))
    print("Prédictions et scoring...")

Train shape : (172975, 755)
Eval shape : (57658, 755)
Nombre de prédicteurs : 755

Entraînement...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[345]	train's binary_logloss: 0.206944	train's auc: 0.864962	eval's binary_logloss: 0.235805	eval's auc: 0.787673
Durée du fit (hh:mm:ss): 0:00:21
Prédictions et scoring...


GPU

In [115]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": ["binary_logloss", "auc"],
    
    "num_leaves": 34,
    "learning_rate": 0.05,
    #"colsample_bytree":0.9497036,
    "max_depth": 8,
    "reg_alpha": 0.041545473,
    "reg_lambda": 0.0735294,
    "min_split_gain": 0.0222415,
    "min_child_weight": 39.3259775,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "device": "gpu",
    "verbose": -1, # Supprime la plupart des warnings lgbm
}
verbose = True

In [116]:
# Conversion en dataset lgbm
lgb_train = lgb.Dataset(X_train.to_pandas(), y_train.to_pandas(), free_raw_data=True)
lgb_eval = lgb.Dataset(X_test.to_pandas(), y_test.to_pandas(), reference=lgb_train, free_raw_data=True)

metrics = {}
if verbose:
    callbacks = [
        lgb.early_stopping(stopping_rounds=50),
        # lgb.log_evaluation(period=50),
        lgb.record_evaluation(metrics),
    ]
    print("Train shape :", X_train.shape)
    print("Eval shape :", X_test.shape)
    print("Nombre de prédicteurs :", len(predictors))
    #print("Nombre de features catégorielles :", len(categorical_features))
    print("\nEntraînement...")
else:
    callbacks = [
        lgb.early_stopping(stopping_rounds=50),
        lgb.record_evaluation(metrics),
    ]
t0_fit = time.time()
lgb_clf = lgb.train(
    params,
    lgb_train,
    num_boost_round=400,  # Correspond à n_estimators max
    valid_sets=[lgb_train, lgb_eval],
    valid_names=["train", "eval"],
    feature_name=predictors,
    #categorical_feature=categorical_features,
    callbacks=callbacks,
)

fit_duration = time.time() - t0_fit
if verbose:
    print("Durée du fit (hh:mm:ss):", format_time(fit_duration))
    print("Prédictions et scoring...")

Train shape : (172975, 755)
Eval shape : (57658, 755)
Nombre de prédicteurs : 755

Entraînement...


LightGBMError: No OpenCL device found